# Import LAMMPS dump

In [1]:
from aiida_lammps.common import parse_trajectory
from collections import namedtuple
import numpy as np
import pymatgen.core as mg
from pymatgen.io.cif import CifWriter

/Users/goldmann/opt/anaconda3/lib/python3.9/site-packages/aiida/orm/groups.py:61: UserWarning: no registered entry point for `Group` so its instances will not be storable.
  warnings.warn(message)  # pylint: disable=no-member
/Users/goldmann/opt/anaconda3/lib/python3.9/site-packages/aiida/orm/groups.py:61: UserWarning: no registered entry point for `AutoGroup` so its instances will not be storable.
  warnings.warn(message)  # pylint: disable=no-member
/Users/goldmann/opt/anaconda3/lib/python3.9/site-packages/aiida/orm/groups.py:61: UserWarning: no registered entry point for `ImportGroup` so its instances will not be storable.
  warnings.warn(message)  # pylint: disable=no-member
/Users/goldmann/opt/anaconda3/lib/python3.9/site-packages/aiida/orm/groups.py:61: UserWarning: no registered entry point for `UpfFamily` so its instances will not be storable.
  warnings.warn(message)  # pylint: disable=no-member


In [9]:
def split_traj(file_location):
    lammps_file = open(file_location, 'r')
    lammps_lines = lammps_file.readlines()
    
    r = range(100000, 2101000, 1000)
    counter = 0

    for line in lammps_lines:
        if "ITEM: TIMESTEP" in line:
            structure_file = open(f'structures/structure_{r[counter]}.lammpstrj', 'w+')
            structure_file.write(line)
            counter = counter + 1
        else:
            structure_file.write(line)

In [10]:
def parse_step(lines, intial_line=0):
    if "ITEM: TIMESTEP" not in lines[0]:
        raise IOError("expected line {} to be TIMESTEP".format(intial_line))
    if "ITEM: NUMBER OF ATOMS" not in lines[2]:
        raise IOError("expected line {} to be NUMBER OF ATOMS".format(intial_line + 2))
    if "ITEM: BOX BOUNDS xy xz yz" not in lines[4]:
        raise IOError(
            "expected line {} to be BOX BOUNDS xy xz yz".format(intial_line + 4)
        )
        # TODO handle case when xy xz yz not present -> orthogonal box
    if "ITEM: ATOMS" not in lines[8]:
        raise IOError("expected line {} to be ATOMS".format(intial_line + 8))
    timestep = int(lines[1])
    number_of_atoms = int(lines[3])

    # each pbc contains two letters <lo><hi> such that:
    # p = periodic, f = fixed, s = shrink wrap, m = shrink wrapped with a minimum value
    pbc = lines[4].split()[6:]

    bounds = [line.split() for line in lines[5:8]]
    bounds = np.array(bounds, dtype=float)
    if bounds.shape[1] == 2:
        bounds = np.append(bounds, np.array([0, 0, 0])[None].T, axis=1)

    xy = bounds[0, 2]
    xz = bounds[1, 2]
    yz = bounds[2, 2]

    xlo = bounds[0, 0] - np.min([0.0, xy, xz, xy + xz])
    xhi = bounds[0, 1] - np.max([0.0, xy, xz, xy + xz])
    ylo = bounds[1, 0] - np.min([0.0, yz])
    yhi = bounds[1, 1] - np.max([0.0, yz])
    zlo = bounds[2, 0]
    zhi = bounds[2, 1]

    super_cell = np.array([[xhi - xlo, xy, xz], [0, yhi - ylo, yz], [0, 0, zhi - zlo]])
    cell = super_cell.T
    field_names = lines[8].split()[2:]
    fields = []
    for i in range(number_of_atoms):
        fields.append(lines[9 + i].split())
    atom_fields = {n: v.tolist() for n, v in zip(field_names, np.array(fields).T)}

    return TRAJ_BLOCK(lines, timestep, number_of_atoms, cell, pbc, atom_fields)

In [33]:


TRAJ_BLOCK = namedtuple(
    "TRAJ_BLOCK", ["lines", "timestep", "natoms", "cell", "pbc", "atom_fields"]
)

trial_file = open('structures/structure_101000.lammpstrj', 'r')
trial_lines = trial_file.readlines()

trial = parse_step(trial_lines)

In [34]:


def create_structure(
    traj_block,
    symbol_field="element",
    position_fields=("x", "y", "z"),
    original_structure=None,
):
    symbols = traj_block.atom_fields[symbol_field]
    positions = np.array(
        [traj_block.atom_fields[f] for f in position_fields], dtype=float
    ).T

    lattice = mg.Lattice(traj_block.cell)
    structure = mg.Structure(lattice, symbols, positions)

    return structure

In [35]:
trial_structure = create_structure(trial)

In [36]:


w = CifWriter(trial_structure)
w.write_file('000000.cif')

In [37]:
traj = 'super_al.lammpstrj'

In [38]:
traj[:-9]

'super_al.'